# Parsing a CSGO demofile
##### Last Updated: June 12, 2020

The `csgo` package allows for the parsing of CSGO demofiles into the Pandas format, which makes for easy analysis. To install the package, clone the Github repository, navigate to the root directory and run `python setup.py install`. Also, make sure you have installed at least [Go](https://golang.org/) version 1.14. 

#### What is a demofile?
Each CSGO match typically generates recordings of the game called a demofile. Every map in a match will generate its own demofile. This demofile contains a serialization of the data transferred between the host (game server) and its clients (players). Data transferred to and from the server occurs at a predefined _tick rate_, which defines when client inputs are resolved with the server. For professional games, the server tick rate is usually 128 ticks per second, meaning each tick represents around 7.8 milliseconds. Client inputs represent player actions, such as movement, attacks or grenade throws. Non-client events also register in the demofile, such as round starts and ends. For a detailed overview of CSGO demofiles, see [1].

Since CSGO demofiles are essentially data transferred between the clients and the game server, the data is simply stored as a text of sequential set of events with no contextual information, such as round or map location. 
Thus, due to the highly unstructured nature of these low level CSGO data streams, akin to log files, performing any complex analytic tasks becomes impossible without modeling this data formally into a useful data structure. The `csgo` package addresses this need.

#### How do I get demofiles?
One of the most common ways to obtain demofiles is through the site [HLTV](www.hltv.org). If you navigate to a match page, an hour or so after the match has concluded, it will often contain a link to the demofiles for that match. The link is a `.rar` compressed directory containing a demo for each map in the match.

# Initializing the parser
Let's consider the demo from the following [ESL Pro League Season 10 Finals](https://www.hltv.org/matches/2338065/astralis-vs-liquid-esl-pro-league-season-10-finals). If we download the compressed demofile directory (450 MB), there are three files: 

- `astralis-vs-liquid-m1-inferno.dem`
- `astralis-vs-liquid-m2-nuke.dem`
- `astralis-vs-liquid-m3-dust2.dem`

In order to parse one of the demofiles, you have to instantiate a `DemoParser` object and pass it the path to the demofile, along with a `match_id`. We will use `match_id = "astralis-vs-liquid-m1-inferno.dem"`. 

In [1]:
from csgo.parser import DemoParser

# Create parser object
# Set log=True above if you want to produce a logfile for the parser
demo_parser = DemoParser(demofile = "astralis-vs-liquid-m1-inferno.dem", match_id = "astralis-vs-liquid-m1-inferno.dem")


# Parse the demofile, output results to dictionary with df name as key
data = demo_parser.parse()

16:17:37 [INFO] Initialized CSGODemoParser with demofile astralis-vs-liquid-m1-inferno.dem
16:17:37 [INFO] Starting CSGO Golang demofile parser, reading in /home/peter/Downloads/examples/astralis-vs-liquid-m1-inferno.dem
16:17:37 [INFO] Running Golang parser from /home/peter/.pyenv/versions/3.8.3/lib/python3.8/site-packages/csgo-0.1-py3.8.egg/csgo/parser/
16:17:53 [INFO] Demofile parsing complete
16:17:53 [INFO] Parsing match events
16:17:53 [INFO] Parsed round end 1
16:17:53 [INFO] Parsed round end official 1
16:17:53 [INFO] Parsed round end 2
16:17:53 [INFO] Parsed round end official 2
16:17:53 [INFO] Parsed round end 3
16:17:53 [INFO] Parsed round end official 3
16:17:53 [INFO] Parsed round end 4
16:17:53 [INFO] Parsed round end official 4
16:17:53 [INFO] Parsed round end 5
16:17:53 [INFO] Parsed round end official 5
16:17:53 [INFO] Parsed round end 6
16:17:53 [INFO] Parsed round end official 6
16:17:53 [INFO] Parsed round end 7
16:17:53 [INFO] Parsed round end official 7
16:17:53 [

## Accessing the parsed data

Then, the information from this map will be contained in a series of Pandas dataframes, each referenced by the following keys:

In [2]:
data.keys()

dict_keys(['Rounds', 'Footsteps', 'Damages', 'Kills', 'BombEvents', 'Grenades'])

#### Rounds

In [3]:
data["Rounds"].head(5)

,MatchId,MapName,RoundNum,StartTick,EndTick,EndCTScore,EndTScore,StartTScore,StartCTScore,RoundWinnerSide,...,RoundLoser,Reason,CTCashSpentTotal,CTCashSpentRound,CTEqVal,TCashSpentTotal,TCashSpentRound,TEqVal,CTRoundType,TRoundType
0,astralis-vs-liquid-m1-inferno.dem,de_inferno,1,1161,13300,0,1,0,0,T,...,8370592,TargetBombed,3600,3600,4600,3400,3400,4400,Pistol,Pistol
1,astralis-vs-liquid-m1-inferno.dem,de_inferno,2,13300,30101,1,1,1,0,CT,...,8370592,CTWin,13050,9450,9450,21750,18350,21400,Half Buy,Full Buy
2,astralis-vs-liquid-m1-inferno.dem,de_inferno,3,30101,47004,2,1,1,1,CT,...,8370592,CTWin,23700,10650,28500,29700,7950,7950,Full Buy,Eco
3,astralis-vs-liquid-m1-inferno.dem,de_inferno,4,47004,54144,3,1,1,2,CT,...,8370592,CTWin,39350,15650,24950,30300,600,1200,Full Buy,None
4,astralis-vs-liquid-m1-inferno.dem,de_inferno,5,54144,72667,3,2,1,3,T,...,8370592,TargetBombed,42150,2800,28350,52000,21700,22700,Full Buy,Full Buy


##### Footsteps

In [4]:
data["Footsteps"].head(5)

,MatchId,MapName,RoundNum,Tick,Second,PlayerID,PlayerName,Team,Side,X,Y,Z,XViz,YViz,ViewX,ViewY,AreaID,AreaName,DistanceBombsiteA,DistanceBombsiteB
0,astralis-vs-liquid-m1-inferno.dem,de_inferno,1,3766,20.351562,76561198066693739,EliGE,Team Liquid,T,-1626.006714,231.920258,-63.968750,94.080262,-742.465254,302.475586,2.977295,1,TSpawn,48,39
1,astralis-vs-liquid-m1-inferno.dem,de_inferno,1,3766,20.351562,76561198001151695,NAF,Team Liquid,T,-1518.342407,363.537506,-66.380058,116.052570,-715.604591,271.785278,8.162842,1,TSpawn,48,39
2,astralis-vs-liquid-m1-inferno.dem,de_inferno,1,3766,20.351562,76561197983956651,Magisk,Astralis,CT,2426.950684,1946.602295,128.031250,921.214425,-392.530144,250.773926,8.574829,91,CTSpawn,18,16
3,astralis-vs-liquid-m1-inferno.dem,de_inferno,1,3767,20.359375,76561197995889730,nitr0,Team Liquid,T,-1650.953857,662.440674,-48.968750,88.989009,-654.603944,295.076294,7.322388,1044,TSpawn,52,43
4,astralis-vs-liquid-m1-inferno.dem,de_inferno,1,3767,20.359375,76561198004854956,dupreeh,Astralis,CT,2263.013672,1964.852905,128.031250,887.757892,-388.805530,245.192871,5.333862,67,CTSpawn,19,15


##### Damages

In [5]:
data["Damages"].head(5)

,MatchId,MapName,RoundNum,Tick,Second,VictimX,VictimY,VictimZ,VictimXViz,VictimYViz,...,AttackerID,AttackerName,AttackerTeam,AttackerSide,AttackerTeamEqVal,HpDamage,KillHpDamage,ArmorDamage,WeaponID,HitGroup
0,astralis-vs-liquid-m1-inferno.dem,de_inferno,1,6063,38.296875,1978.013184,173.367020,160.031250,829.594527,-754.414894,...,76561197999004010,Stewie2K,Team Liquid,T,4400,12,12,6,Glock,Head
1,astralis-vs-liquid-m1-inferno.dem,de_inferno,1,6504,41.742188,2113.371338,372.447052,160.031250,857.218640,-713.786316,...,76561198066693739,EliGE,Team Liquid,T,4400,11,11,6,Glock,Head
2,astralis-vs-liquid-m1-inferno.dem,de_inferno,1,6524,41.898438,2117.859131,125.695236,246.534744,858.134517,-764.143829,...,76561197983956651,Magisk,Astralis,CT,4600,20,20,9,USP,Chest
3,astralis-vs-liquid-m1-inferno.dem,de_inferno,1,6579,42.328125,2067.031250,-399.005859,256.031250,847.761480,-871.225686,...,76561197999004010,Stewie2K,Team Liquid,T,4400,11,11,6,Glock,Stomach
4,astralis-vs-liquid-m1-inferno.dem,de_inferno,1,6612,42.585938,2268.382812,199.705856,128.736832,888.853635,-749.039621,...,76561197983956651,Magisk,Astralis,CT,4600,15,15,7,USP,Head


##### Kills

In [6]:
data["Kills"].head(5)

,MatchId,MapName,RoundNum,Tick,Second,VictimX,VictimY,VictimZ,VictimXViz,VictimYViz,...,AttackerSide,AttackerTeamEqVal,AssisterID,AssisterName,AssisterTeam,AssisterSide,WeaponID,IsWallshot,IsFlashed,IsHeadshot
0,astralis-vs-liquid-m1-inferno.dem,de_inferno,1,6629,42.718750,2070.210938,-400.019836,255.738281,848.410395,-871.432620,...,T,4400,0,NA,NA,NA,Glock,0,0,1
1,astralis-vs-liquid-m1-inferno.dem,de_inferno,1,6702,43.289062,2236.474121,358.814270,134.983994,882.341657,-716.568516,...,T,4400,0,NA,NA,NA,Glock,0,0,1
2,astralis-vs-liquid-m1-inferno.dem,de_inferno,1,10756,74.960938,2550.814209,1377.042847,160.031250,946.492696,-508.766766,...,T,4400,76561198016255205,Twistzz,Team Liquid,T,USP,0,0,1
3,astralis-vs-liquid-m1-inferno.dem,de_inferno,1,12021,84.843750,-349.312500,-305.413849,192.031250,354.630102,-852.125275,...,T,3750,0,NA,NA,NA,Glock,0,0,1
4,astralis-vs-liquid-m1-inferno.dem,de_inferno,2,19776,50.593750,64.399773,1463.703857,176.031250,439.061178,-491.080845,...,CT,9950,0,NA,NA,NA,Deagle,0,0,0


##### Bomb Events

In [7]:
data["BombEvents"].head(5)

,MatchId,MapName,RoundNum,Tick,Second,PlayerID,PlayerName,Team,AreaID,BombSite,EventType
0,astralis-vs-liquid-m1-inferno.dem,de_inferno,1,7411,48.828125,76561198016255205,Twistzz,Team Liquid,157,A,Plant
1,astralis-vs-liquid-m1-inferno.dem,de_inferno,5,66780,98.718750,76561198066693739,EliGE,Team Liquid,2086,B,Plant
2,astralis-vs-liquid-m1-inferno.dem,de_inferno,7,102976,96.789062,76561198001151695,NAF,Team Liquid,829,B,Plant
3,astralis-vs-liquid-m1-inferno.dem,de_inferno,9,135050,64.570312,76561198001151695,NAF,Team Liquid,157,A,Plant
4,astralis-vs-liquid-m1-inferno.dem,de_inferno,10,149050,63.359375,76561198066693739,EliGE,Team Liquid,157,A,Plant


##### Grenades

In [8]:
data["Grenades"].head(5)

,MatchId,MapName,RoundNum,Tick,Second,PlayerID,PlayerName,Team,Side,X,Y,Z,XViz,YViz,AreaID,AreaName,GrenadeType
0,astralis-vs-liquid-m1-inferno.dem,de_inferno,1,4982,29.851562,76561197995889730,nitr0,Team Liquid,T,995.819031,461.206726,205.201797,629.146741,-695.672097,21,Middle,Flash
1,astralis-vs-liquid-m1-inferno.dem,de_inferno,1,5225,31.750000,76561198010511021,gla1ve,Astralis,CT,373.511475,2258.871826,383.608948,502.145199,-328.801668,1933,BombsiteB,Flash
2,astralis-vs-liquid-m1-inferno.dem,de_inferno,1,5237,31.843750,76561197999004010,Stewie2K,Team Liquid,T,874.956848,16.081377,94.751358,604.480989,-786.514005,479,SecondMid,Decoy
3,astralis-vs-liquid-m1-inferno.dem,de_inferno,1,5770,36.007812,76561197995889730,nitr0,Team Liquid,T,1428.454956,484.867035,259.739105,717.439787,-690.843462,31,TopofMid,Flash
4,astralis-vs-liquid-m1-inferno.dem,de_inferno,1,5922,37.195312,76561197995889730,nitr0,Team Liquid,T,2295.472168,723.563904,148.717834,894.382075,-642.129816,69,BombsiteA,Smoke


## References
[1] Bednárek, David, et al. "Player Performance Evaluation in Team-Based First-Person Shooter eSport." International Conference on Data Management Technologies and Applications. Springer, Cham, 2017.